In [3]:
import csv
import json
import requests
from time import sleep

In [4]:
def follow_artist(artist_url, max_degree=None, degree=0, from_artist_id=None, from_release_id=None, reverse=False, relationship=None, role=''):
    if artist_url == '':
        return
    while True:
        sleep(2)
        r = requests.get(artist_url, headers={'User-agent': 'FooBarApp/3.0'})
        if r.status_code == 200 or r.status_code == 404:
            break
    response = json.loads(r.text)
    if 'name' not in response or r.status_code == 404: # artist not found
        return
    artist_id = 'artist/' + artist_url[artist_url.rfind('/') + 1:]
    if artist_url not in artists:
        artists_writer.writerow({'id:ID': artist_id, 'name': response['name'].replace('"', "'"), 'url': response['uri'], ':LABEL': 'ARTIST'})
        artists_csvfile.flush()
    from_id = from_artist_id if from_artist_id else from_release_id
    if from_id:
        if reverse:
            relationships_writer.writerow({':START_ID': artist_id, 'role': '', ':END_ID': from_id, ':TYPE': relationship})
        else:
            relationships_writer.writerow({':START_ID': from_id, 'role': '', ':END_ID': artist_id, ':TYPE': relationship})
    relationships_csvfile.flush()

    if artist_url in artists:
        return # I've been here before
    artists[artist_url] = response['name']
    print(f'{len(artists)}. {artists[artist_url]} {artist_url}')
    if degree >= max_degree:
        return
    for alias in response.get('members', range(0, 0)):
        follow_artist(alias['resource_url'], max_degree, degree + 1, from_artist_id=artist_id, relationship='PLAYED_IN', reverse=True)
    for alias in response.get('aliases', range(0, 0)):
        follow_artist(alias['resource_url'], max_degree, degree + 1, from_artist_id=artist_id, relationship='AKA')
    for group in response.get('groups', range(0, 0)):
        follow_artist(group['resource_url'], max_degree, degree + 1, from_artist_id=artist_id, relationship='PLAYED_IN')

    while True:
        sleep(2)
        r = requests.get(response['releases_url'], headers={'User-agent': 'FooBarApp/3.0'})
        if r.status_code == 200:
            break
    response_ = json.loads(r.text)
    while True:
        for release in response_['releases']:
            release_url = release['resource_url']
            
            while True:
                sleep(2)
                r = requests.get(release_url, headers={'User-agent': 'FooBarApp/3.0'})
                if r.status_code == 200:
                    break
            response__ = json.loads(r.text)
            release_id = 'release/' + release_url[release_url.rfind('/') + 1:]
            if release_url not in releases:
                releases_writer.writerow({'id:ID': release_id, 'name': release['title'].replace('"', "'"), 'url': response__['uri'], ':LABEL': 'RELEASE'})
                releases_csvfile.flush()
            relationships_writer.writerow({':START_ID': artist_id, 'role': '', ':END_ID': release_id, ':TYPE': 'APPEARS_ON'})
            relationships_csvfile.flush()

            if release_url in releases:
                continue # I've been here before
            releases[release_url] = release['title']
            for track in response__.get('tracklist', range(0, 0)):
                for artist in track.get('artists', range(0, 0)):
                    follow_artist(artist['resource_url'], max_degree, degree + 1, from_release_id=release_id, role=artist.get('role', ''), relationship='APPEARS_ON', reverse=True)
                for extraartist in track.get('extraartists', range(0, 0)):
                    follow_artist(extraartist['resource_url'], max_degree, degree + 1, from_release_id=release_id, role=extraartist.get('role', ''), relationship='APPEARS_ON', reverse=True)

        if 'next' not in response_['pagination']['urls']:
            break
        while True:
            sleep(2)
            r = requests.get(response_['pagination']['urls']['next'], headers={'User-agent': 'FooBarApp/3.0'})
            if r.status_code == 200:
                break
        response_ = json.loads(r.text)

In [5]:
artists = {}
releases = {}

artists_csvfile = open('artists-header.csv', 'wt', newline='')
fieldnames = ['id:ID', 'name', 'url', ':LABEL']
artists_writer = csv.DictWriter(artists_csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
artists_writer.writeheader()
artists_csvfile.close()
artists_csvfile = open('artists.csv', 'wt', newline='')
artists_writer = csv.DictWriter(artists_csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)

releases_csvfile = open('releases-header.csv', 'wt', newline='')
fieldnames = ['id:ID', 'name', 'url', ':LABEL']
releases_writer = csv.DictWriter(releases_csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
releases_writer.writeheader()
releases_csvfile.close()
releases_csvfile = open('releases.csv', 'wt', newline='')
releases_writer = csv.DictWriter(releases_csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)

relationships_csvfile = open('relationships-header.csv', 'wt', newline='')
fieldnames=[':START_ID', 'role', ':END_ID', ':TYPE']
relationships_writer = csv.DictWriter(relationships_csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
relationships_writer.writeheader()
relationships_csvfile.close()
relationships_csvfile = open('relationships.csv', 'wt', newline='')
relationships_writer = csv.DictWriter(relationships_csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)

In [ ]:
follow_artist('https://api.discogs.com/artists/25058', max_degree=2)

1. MF Doom https://api.discogs.com/artists/25058
2. King Ghidra https://api.discogs.com/artists/113391
3. Viktor Vaughn https://api.discogs.com/artists/143795
4. Metal Fingers https://api.discogs.com/artists/188739
5. Zev Love X https://api.discogs.com/artists/207226
6. The Super Villain https://api.discogs.com/artists/528596
7. Sci.Fly https://api.discogs.com/artists/661049
8. Daniel Dumile https://api.discogs.com/artists/701676
9. Daniel Thompson (4) https://api.discogs.com/artists/1500203
10. The Metal Finger Villain https://api.discogs.com/artists/4556570
11. King Dumile https://api.discogs.com/artists/4725188
12. The Emef https://api.discogs.com/artists/4906755
13. Mr. Fantastik https://api.discogs.com/artists/117240
14. Hassan Chop https://api.discogs.com/artists/117242
15. E. Mason https://api.discogs.com/artists/113398
16. Biolante https://api.discogs.com/artists/117234
17. Gigan https://api.discogs.com/artists/117235
18. Stacy Epps https://api.discogs.com/artists/186438
19. ID

151. J-Live https://api.discogs.com/artists/5953
152. Athletic Mic League https://api.discogs.com/artists/238292
153. OneManArmy https://api.discogs.com/artists/385864
154. Concise Kilgore https://api.discogs.com/artists/349313
155. RJD2 https://api.discogs.com/artists/336190
156. Jakki Da Motamouth https://api.discogs.com/artists/82930
157. Cyne https://api.discogs.com/artists/93025
158. Evolver (5) https://api.discogs.com/artists/152119
159. Shaman (4) https://api.discogs.com/artists/868627
160. Ravage (2) https://api.discogs.com/artists/868628
161. B-Wyze (2) https://api.discogs.com/artists/868625
162. KMD https://api.discogs.com/artists/132998
163. Substance Abuse https://api.discogs.com/artists/117896
164. Brand Nubian https://api.discogs.com/artists/30011
165. Jon Doe (2) https://api.discogs.com/artists/204593
166. DJ Food Stamp https://api.discogs.com/artists/1570865
167. Prince Paul https://api.discogs.com/artists/13247
168. Madvillain https://api.discogs.com/artists/157763
169

298. L'Atelier https://api.discogs.com/artists/164744
299. Natural Elements https://api.discogs.com/artists/38249
300. Klub des Loosers https://api.discogs.com/artists/203595
301. Jonathan Lambert https://api.discogs.com/artists/591022
302. Un Japonais Fou https://api.discogs.com/artists/504089
303. Dizzee Rascal https://api.discogs.com/artists/78659
304. Wiley (2) https://api.discogs.com/artists/197311
305. Max Tundra https://api.discogs.com/artists/418
306. TTC https://api.discogs.com/artists/28559
307. dDamage https://api.discogs.com/artists/71821
308. The Other People Place https://api.discogs.com/artists/20583
309. Gang Starr https://api.discogs.com/artists/29058
310. Jadakiss https://api.discogs.com/artists/37909
311. Craig G https://api.discogs.com/artists/44392
312. Rockwilder https://api.discogs.com/artists/62572
313. Funky Fresh Few https://api.discogs.com/artists/14725
314. Q Ball & Curt Cazal https://api.discogs.com/artists/1769553
315. The Narcissists https://api.discogs.c

447. Wildchild (2) https://api.discogs.com/artists/107705
448. Vinia Mojica https://api.discogs.com/artists/85870
449. John Berruggio https://api.discogs.com/artists/1853607
450. Subroc https://api.discogs.com/artists/150588
451. Quasimoto https://api.discogs.com/artists/24208
452. Passage https://api.discogs.com/artists/55553
453. DJ Signify https://api.discogs.com/artists/85562
454. Express Rising https://api.discogs.com/artists/210677
455. Zero 7 https://api.discogs.com/artists/26647
456. Controller 7 https://api.discogs.com/artists/79702
457. Charizma https://api.discogs.com/artists/34219
458. Peanut Butter Wolf https://api.discogs.com/artists/16393
459. Reaching Quiet https://api.discogs.com/artists/32294
460. EL-P https://api.discogs.com/artists/2202
461. Orchestral Manoeuvres In The Dark https://api.discogs.com/artists/9462
462. Sixtoo https://api.discogs.com/artists/43174
463. Dr. Dooom https://api.discogs.com/artists/102148
464. Dreas https://api.discogs.com/artists/219542
465

597. Dj Breaque https://api.discogs.com/artists/1750016
598. Aesop Rock https://api.discogs.com/artists/28104
599. State Of The Union (5) https://api.discogs.com/artists/7764850
600. The Planets https://api.discogs.com/artists/115819
601. Pase Rock https://api.discogs.com/artists/214535
602. 1 Universe https://api.discogs.com/artists/7764770
603. Grap Luva https://api.discogs.com/artists/37253
604. Prince Po https://api.discogs.com/artists/118648
605. Rob Swift https://api.discogs.com/artists/9840
606. Daedalus https://api.discogs.com/artists/192216
607. Alias (3) https://api.discogs.com/artists/48536
608. The Casual Brothers https://api.discogs.com/artists/240489
609. Shadow Huntaz https://api.discogs.com/artists/153840
610. Copywrite https://api.discogs.com/artists/32215
611. Non Phixion https://api.discogs.com/artists/82700
612. Qvel https://api.discogs.com/artists/3259747
613. Maker https://api.discogs.com/artists/219089
614. Staahr https://api.discogs.com/artists/7764769
615. Dabr

...

4359. The Roze https://api.discogs.com/artists/4560863
4360. Kruse https://api.discogs.com/artists/348127
4361. Nick Classic https://api.discogs.com/artists/4560870
4362. DJ Mukeccho https://api.discogs.com/artists/6694357
4363. DJ Nice https://api.discogs.com/artists/103490
4364. Koron https://api.discogs.com/artists/1086076
4365. Nutso https://api.discogs.com/artists/267378
4366. Hustletron https://api.discogs.com/artists/23362
4367. Ann Peebles https://api.discogs.com/artists/80089
4368. Soul Survivors https://api.discogs.com/artists/127616
4369. Jslikk https://api.discogs.com/artists/6399556
4370. Donald Mayhem https://api.discogs.com/artists/5839479
4371. Infinito 2017 https://api.discogs.com/artists/505845
4372. Thaione Davis https://api.discogs.com/artists/239395
4373. Zoser https://api.discogs.com/artists/626157
4374. Qwazaar https://api.discogs.com/artists/283980
4375. Czar https://api.discogs.com/artists/949455
4376. Hellsent https://api.discogs.com/artists/861674
4377. Adad 

4503. Sound Thoughts https://api.discogs.com/artists/972221
4504. Pijall https://api.discogs.com/artists/218435
4505. Mattip https://api.discogs.com/artists/218433
4506. Triple Threat (3) https://api.discogs.com/artists/287230
4507. Dujeous? https://api.discogs.com/artists/234463
4508. 3rd Degree (2) https://api.discogs.com/artists/295668
4509. Ronnie Cash https://api.discogs.com/artists/3526090
4510. Yak Ballz https://api.discogs.com/artists/40432
4511. DJ Heavy (3) https://api.discogs.com/artists/3961441
4512. Jayem (4) https://api.discogs.com/artists/1532746
4513. TelepathMath https://api.discogs.com/artists/1772422
4514. Paradime https://api.discogs.com/artists/116149
4515. DJ Rectangle https://api.discogs.com/artists/71214
4516. EP The Energy Provider https://api.discogs.com/artists/6485581
4517. Lacksidaisycal https://api.discogs.com/artists/22815
4518. Johnny Lawrence https://api.discogs.com/artists/428330
4519. Roast (2) https://api.discogs.com/artists/6485582
4520. Uncle ILL h

4647. Erica "Miss America" https://api.discogs.com/artists/1082283
4648. PZ (5) https://api.discogs.com/artists/1138534
4649. Chip Fu https://api.discogs.com/artists/258664
4650. Tyren Corington https://api.discogs.com/artists/1082282
4651. Vanessa Hill https://api.discogs.com/artists/1082278
4652. Zo! https://api.discogs.com/artists/334903
4653. Victor St. Clair https://api.discogs.com/artists/1166005
4654. Chocolate Puma https://api.discogs.com/artists/8095
4655. St Michel https://api.discogs.com/artists/551259
4656. Cesar Comanche https://api.discogs.com/artists/341389
4657. Chaundon https://api.discogs.com/artists/326695
4658. Big Dho https://api.discogs.com/artists/313356
4659. The Away Team (2) https://api.discogs.com/artists/326696
4660. Undefined https://api.discogs.com/artists/474822
4661. Slopfunkdust https://api.discogs.com/artists/1099388
4662. Ark (4) https://api.discogs.com/artists/370173
4663. Carvin Haggins https://api.discogs.com/artists/182497
4664. Ivan "Orthodox" Ba

4790. Malene Younglao https://api.discogs.com/artists/485561
4791. Jamaram https://api.discogs.com/artists/512131
4792. La Cedille https://api.discogs.com/artists/300723
4793. Da Blingz https://api.discogs.com/artists/5547642
4794. Pluto (32) https://api.discogs.com/artists/3571379
4795. Madalena João https://api.discogs.com/artists/3560052
4796. Navigators (5) https://api.discogs.com/artists/1979548
4797. Aux 88 https://api.discogs.com/artists/786
4798. Selan https://api.discogs.com/artists/166627
4799. Alix Alvarez https://api.discogs.com/artists/35108
4800. Terri Wells https://api.discogs.com/artists/88595
4801. Vikter Duplaix https://api.discogs.com/artists/5366
4802. Kings Of Soul https://api.discogs.com/artists/28334
4803. Joey Negro https://api.discogs.com/artists/1825
4804. Ivana Santilli https://api.discogs.com/artists/107116
4805. Tim Motzer https://api.discogs.com/artists/137919
4806. Troy Simms https://api.discogs.com/artists/445450
4807. William Allen https://api.discogs.c

4933. Mervin A. Jenkins https://api.discogs.com/artists/1043627
4934. Mischa "Big Dho" Burgess https://api.discogs.com/artists/3187029
4935. KeKe https://api.discogs.com/artists/663445
4936. Tyesha https://api.discogs.com/artists/663446
4937. The Homie https://api.discogs.com/artists/663444
4938. Nick Carter (4) https://api.discogs.com/artists/739123
4939. Eternia https://api.discogs.com/artists/79766
4940. Frequency (5) https://api.discogs.com/artists/273584
4941. Quartermaine https://api.discogs.com/artists/365254
4942. Raks One https://api.discogs.com/artists/324873
4943. Presto (2) https://api.discogs.com/artists/295680
4944. Isaac Jones https://api.discogs.com/artists/431771
4945. Roddy Rod https://api.discogs.com/artists/184987
4946. The Sound Defects https://api.discogs.com/artists/3365486
4947. Kypski https://api.discogs.com/artists/286497
4948. Humpty Hump https://api.discogs.com/artists/119155
4949. Pete Philly & Perquisite https://api.discogs.com/artists/359702
4950. Brian (

In [ ]:
artists_csvfile.close()
releases_csvfile.close()
relationships_csvfile.close()